<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module3/M3_1/General_Framework_LP_Pyomo_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Framework for LP models in Pyomo (answers)
**OPIM 5641: Business Decision Modeling - Dept. of Operations and Information Management - University of Connecticut**

-------------------------------
The purpose of this script is to give you a general format for coding up LP models in Pyomo. If you can repeat the major elements in this script then you will be on your way toward optimization modeling greatness! Of course, we will use chairs, desks and tables here for our example.

# Sample Word Problem


*Section 9.2 (Powell) - Chairs, Desks and Tables*

**Problem Description:**
Veerman Furniture Company makes three kinds of ofﬁce furniture: chairs, desks, and tables. Each product requires some labor in the parts fabrication department, the assembly department, and the shipping department. The furniture is sold through a regional distributor, who has estimated the maximum potential sales for each product in the coming quarter. Finally,the accounting department has provided some data showing the proﬁt contributions on each product.The decision problem isto determine the product mix—that is, to maximize Veerman’s proﬁt for the quarter by choosing production quantities for the chairs,desks,and tables.

The following data summarizes the parameters of the problem:

Department | Chairs | Desks | Tables | Hours Available
--- | --- | --- | --- | ---
Fabrication | 4 | 6 | 2 | 1,850
Assembly | 3 | 5 | 7 | 2,400
Shipping | 3 | 2 | 4 | 1,500
--------------------------------------------------------------------------------
Demand Potential | 360 | 300 | 100 |
Profit (USD) | 15 | 24 | 18 |

**Define the Objective Function**

$Profit = 15C + 24D + 18T$

**Write the Constraints**

$Max(Z) = 15C + 24D + 18T$

subject to:
* $4C + 6D + 2T <= 1,850$
* $3C + 5D + 7T <= 2,400$
* $3C + 2D + 4T <= 1,500$ 
* $C        <= 360$ 
* $D      <=300$ 
* $T <=100$

Great! Now that your problem is defined - go code it up and solve it.

## Import modules (download Pyomo)
You must do this everytime! During class, we may use different solvers depending on the type of optimization problem we are trying to solve. 

In [ ]:
# import modules

# this makes beautiful plots using pylab (matplotlib)
%matplotlib inline
from pylab import * # * means import ALL NAME SPACES

# useful modules for downloading pyomo onto Colab
import shutil
import sys
import os.path

# install pyomo if it doesn't exist
if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

# install the 'cbc' solve if it doesn't exist
if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

# make sure 'cbc' is the solver that we will invoke later on
assert(shutil.which("cbc") or os.path.isfile("cbc"))

# import ALL NAMESPACES (variables) from pyomo
from pyomo.environ import *

### A note about the asterisk...

**Source:** https://stackoverflow.com/questions/2360724/what-exactly-does-import-import

This practice (of importing * into the current namespace) is however discouraged because it

* provides the opportunity for namespace collisions (say if you had a variable name pi prior to the import)
* may be inefficient

The alternative way to use `pyomo` is as follows (from p.3 in 'Pyomo - Optimization Modeling in Python (3rd Edition)'



```
import pyomo.environ as pyo

model = pyo.ConcreteModel()
model.x_1 = pyo.Var(within=pyo.NonNegativeReals)
model.x_2 = pyo.Var(within=pyo.NonNegativeReals)
model.obj = pyo.Objective(expr=model.x_1 + 2*model.x_2)
model.con1 = pyo.Constraint(expr=3*model.x_1 + 4*model.x_2)
model.con2 = pyo.Constraint(expr=2*model.x_1 + 5*model.x_2)
```

And so on... but honestly, I've never really had an issue with overwriting namespaces. But if you want to code like this for your homework, feel free. Not required.



# ConcreteModel()
In our class, we will typically use the `ConcreteModel()` instead of `AbstractModel()` because it's easier to work with

In [ ]:
# you will always need to write this code
model = ConcreteModel()

### A note about $\LaTeX$ 
As a best practice, why not write our the formulas in LaTeX so it's easy for a layman to read your logic? :)

# Declare Decision Variables
You get to name the variables and choose the domain of the decision variables. At this point, you could force an integer solution if you wanted to...

* bounds = A function (or Python object) that gives a (lower,upper) bound pair for the variable
* domain = A set that is a super-set of the values the variable can take on.
* initialize = A function (or Python object) that gives a starting value for the variable; this is particularly important for non-linear models


See here for more: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html

In [ ]:
model.chairs = Var(domain=NonNegativeReals) # could try NonNegativeIntegers here too!
model.desks = Var(domain=NonNegativeReals) # d for desks
model.tables = Var(domain=NonNegativeReals) # t for tables

#  Objective Function
You can choose to `maximize` or `minimize` the objective function - remember, you can't do both! Given these are linear programs, no exponents, absolute value or square roots etc. allowed!

See more here: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html

In [ ]:
# declare objective
model.profit = Objective(
                      expr = 15*model.chairs + 24*model.desks + 18*model.tables, # values come from the table
                      sense = maximize) 


You can name your objective function anything you want - you don't have to call it `model.profit`! You can generalize and call it `model.z` or `model.Obj`. 

# Constraints
You have the freedom to name your constraints ANYTHING you want! This approach works, but you can try renaming `model.Constraint1` to `model.FabricationConstraint` and try to run it, too.

See more here: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html

In [ ]:
# write the constraints one by one, don't forget LHS vs. RHS
# it's OK to have mixed constraints (LHS <= RHS vs. LHS >= RHS)!!!
model.constraint1 = Constraint(expr = 4*model.chairs + 6*model.desks + 2*model.tables <= 1850) # fabrication hours
model.constraint2 = Constraint(expr = 3*model.chairs + 5*model.desks + 7*model.tables <= 2400) # assembly hours
model.constraint3 = Constraint(expr = 3*model.chairs + 2*model.desks + 4*model.tables <= 1500) # shipping
model.constraint4 = Constraint(expr = model.chairs <= 360) # c demand
model.constraint5 = Constraint(expr = model.desks <= 300) # d demand
model.constraint6 = Constraint(expr = model.tables <= 100) # t demand

Note: you can name your constraints anything you want!

# [optional] Pretty Print (pprint)
Optional, but you should probably run this to check your work! Especially if you get an error.

In [ ]:
# show the model you've created
model.pprint()

3 Var Declarations
    chairs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    desks : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    tables : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 15*chairs + 24*desks + 18*tables

6 Constraint Declarations
    constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                          : Upper  : Active
        None :  -Inf : 4*chairs + 6*desks + 2*tables : 1850.0 :   True
    constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body   

# Solve!
Now the momennt of truth - two parts you need to pay attention to - the name of the solver (`cbc`) and the path to the solver (`executable='/usr/bin/cbc'` you specified this at the top of the script).

In [ ]:
#            solver          path the solver
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8400.0
  Upper bound: 8400.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s

After you run the model, it will have a 'termination message' that reads `Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.`

If it says this message, you solved! If it doesn't you probably have a bug in your code (invalid constraints, a typo in your objective function, etc.)

# Inspect the Results

## Objective Function
This the optimal solution

In [ ]:
model.profit # don't forget the round brackets!

In [ ]:
model.profit() # this is better! with round brackets!

8400.0

In [ ]:
print("Profit = ", model.profit(), " per week")

Profit =  8400.0  per week


The use of the `print()` statments makes your output look nice - always do this!!!

## Decision Variables
Note how we need `()` after the variable - lots of students forget this and get a bug!

In [ ]:
print("Chairs = ", model.chairs(), " units per week")
print("Desks = ", model.desks(), " units per week")
print("Tables = ", model.tables(), " units per week")

Chairs =  0.0  units per week
Desks =  275.0  units per week
Tables =  100.0  units per week


## Constraints
Again, don't forget `()` and look for any BINDING constraints where `LHS = RHS`.

In [ ]:
print("Fabrication = ", model.constraint1(), "hours")
print("Assembly = ", model.constraint2(), "hours")
print("Shipping = ", model.constraint3(), "hours")
print("Chairs = ", model.constraint4(), " units per week")
print("Desks = ", model.constraint5(), " units per week")
print("Tables = ", model.constraint6(), " units per week")

Fabrication =  1850.0 hours
Assembly =  2075.0 hours
Shipping =  950.0 hours
Chairs =  0.0  units per week
Desks =  275.0  units per week
Tables =  100.0  units per week


# All done!
Congrats! If you can repeat this (even just the bold headers), you are on your way to optimization modeling greatness. You may also choose to time your script or perform a sensitivty analysis on any binding constraints.